In [1]:
import pandas as pd


# Definir las rutas de los archivos
ruta_clientes = './datos/clientes.csv'
ruta_productos = './datos/productos.csv'
ruta_boletas = './datos/boletas.csv'
ruta_detalle = './datos/detalle.csv'

# Cargar los datos en DataFrames
df_clientes = pd.read_csv(ruta_clientes)
df_productos = pd.read_csv(ruta_productos)
df_boletas = pd.read_csv(ruta_boletas)
df_detalle = pd.read_csv(ruta_detalle)

1. Para  nuestros  datos  de  supermercado, crear  una  lista  de  los clientes, con el gasto hecho en compras de cada uno. Si no ha comprado, debe aparecer con un 0

In [2]:
# Unir los DataFrames
df_compras = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')

# Necesitamos también los precios de los productos para calcular el gasto total
df_compras = pd.merge(df_compras, df_productos, left_on='codigo_producto', right_on='codigo')

# Calcular el gasto de cada cliente
df_compras['gasto'] = df_compras['cantidad'] * df_compras['precio']
gasto_por_cliente = df_compras.groupby('rut_cliente')['gasto'].sum()

# Convertir la serie a DataFrame
gasto_por_cliente = gasto_por_cliente.reset_index()

# Unir con el DataFrame de clientes para incluir a los que no han hecho compras
df_clientes_gasto = pd.merge(df_clientes, gasto_por_cliente, left_on='rut', right_on='rut_cliente', how='left')

# Reemplazar los NaN por 0
df_clientes_gasto['gasto'] = df_clientes_gasto['gasto'].fillna(0)

# Mostrar el DataFrame
print(df_clientes_gasto)

           rut             nombre                direccion        comuna  \
0   11223344-4      Tomás Ramírez                      NaN         Arica   
1   11223344-5        Laura Rojas        Calle Virtual 567   Antofagasta   
2   12345678-9         Juan Perez          Calle Falsa 123      Santiago   
3   23456789-1    Pedro Rodríguez                      NaN    Valparaíso   
4   33445566-5      Camila Torres    Calle Cibernética 456           NaN   
5   33445566-7         Mateo Diaz                      NaN  Viña del Mar   
6   34567890-2     Carla González      Calle Inventada 789           NaN   
7   45678901-3     Diego Martínez       Avenida Irreal 987     La Serena   
8   55667788-6      Matias Castro   Avenida Innovación 321           NaN   
9   55667788-9  Valentina Herrera  Avenida Tecnológica 123           NaN   
10  56789012-4          Ana Lopez                      NaN           NaN   
11  67890123-5    Luisa Fernández       Calle Ficticia 321      Rancagua   
12  77889900

2. Crea un DataFrame con datos de ventas por producto y región. Utiliza multi-índices para representar la información por año y trimestre. Luego, calcula la suma de ventas por trimestre para cada producto.

In [3]:
import pandas as pd

# Cambiar el nombre de la columna 'nombre' en df_productos y df_clientes
df_productos.rename(columns={'nombre': 'nombre_producto'}, inplace=True)
df_clientes.rename(columns={'nombre': 'nombre_cliente'}, inplace=True)

# Unir los DataFrames
df_ventas = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')
df_ventas = pd.merge(df_ventas, df_productos, left_on='codigo_producto', right_on='codigo')
df_ventas = pd.merge(df_ventas, df_clientes, left_on='rut_cliente', right_on='rut')

# Extraer el año y el trimestre de la fecha
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
df_ventas['Año'] = df_ventas['fecha'].dt.year
df_ventas['Trimestre'] = df_ventas['fecha'].dt.quarter

# Crear un multi-índice con el producto, la región (comuna), el año y el trimestre
df_ventas.set_index(['nombre_producto', 'comuna', 'Año', 'Trimestre'], inplace=True)

# Calcular la suma de ventas por trimestre para cada producto en cada región
ventas_por_trimestre = df_ventas.groupby(level=['nombre_producto', 'comuna', 'Año', 'Trimestre'])['cantidad'].sum()

# Mostrar el resultado
print(ventas_por_trimestre)

nombre_producto   comuna        Año   Trimestre
Aceite de cocina  Santiago      2023  4             6
Agua mineral      Iquique       2023  4             3
                  La Serena     2023  4             3
Arroz             Santiago      2023  4             9
Azúcar            Iquique       2023  4             7
                  Puerto Montt  2023  4             3
                  Santiago      2023  4             2
Café              Antofagasta   2023  4             4
                  Valparaíso    2023  4             8
Fideos            Antofagasta   2023  4             5
                  Concepción    2023  4             5
Galletas          Antofagasta   2023  4             7
Harina            Antofagasta   2023  4             1
                  Concepción    2023  4             1
                  Iquique       2023  4             3
                  Santiago      2023  4             2
Helado            Antofagasta   2023  4             4
                  Temuco        20

3. Tienes dos DataFrames: uno con información de clientes (ID, nombre, dirección) y otro con sus pedidos (ID del cliente, producto, cantidad). Realiza una combinación de datos para obtener un DataFrame que muestre el nombre del cliente, su dirección y los productos pedidos por cada cliente.

In [4]:
import pandas as pd

# Primero, combina df_boletas y df_detalle
df_boletas_detalle = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')

# Luego, combina el resultado con df_productos
df_boletas_detalle_productos = pd.merge(df_boletas_detalle, df_productos, left_on='codigo_producto', right_on='codigo')

# Finalmente, combina el resultado con df_clientes
df_combinado = pd.merge(df_clientes, df_boletas_detalle_productos, left_on='rut', right_on='rut_cliente')


# Puedes seleccionar solo las columnas que necesitas
df_combinado = df_combinado[['nombre_cliente', 'direccion', 'nombre_producto', 'cantidad']]

#verificamos las columnas
#print("Columnas en df_combinado:", df_combinado.columns)

# Mostrar el resultado
print(df_combinado)

       nombre_cliente                direccion   nombre_producto  cantidad
0         Laura Rojas        Calle Virtual 567              Café         4
1         Laura Rojas        Calle Virtual 567          Galletas         7
2         Laura Rojas        Calle Virtual 567           Pescado         4
3         Laura Rojas        Calle Virtual 567            Helado         4
4         Laura Rojas        Calle Virtual 567            Fideos         5
5         Laura Rojas        Calle Virtual 567            Harina         1
6          Juan Perez          Calle Falsa 123             Arroz         3
7          Juan Perez          Calle Falsa 123  Aceite de cocina         6
8          Juan Perez          Calle Falsa 123            Azúcar         2
9          Juan Perez          Calle Falsa 123             Arroz         6
10         Juan Perez          Calle Falsa 123            Harina         2
11         Juan Perez          Calle Falsa 123             Leche         1
12    Pedro Rodríguez    

 4. Para una tarde de bingo se usarán tablas generadas con las siguientes reglas:
•      Los 5 números en cada columna son aleatorios ordenados no repetidos de acuerdo al rango asignado por columna.
•      La casilla central de la tabla tiene valor marcado cero o ‘libre’. Realice un programa para generar una tabla de bingo

In [5]:
import numpy as np
import pandas as pd

# Definir los rangos para cada columna
rangos = [(1, 15), (16, 30), (31, 45), (46, 60), (61, 75)]

# Crear una lista vacía para almacenar las columnas
columnas = []

# Para cada rango
for rango in rangos:
    # Generar 5 números aleatorios no repetidos y ordenarlos
    numeros = np.random.choice(range(rango[0], rango[1] + 1), size=5, replace=False)
    numeros.sort()
    # Añadir los números a las columnas
    columnas.append(numeros)

# Crear la tabla de bingo como un DataFrame de strings
tabla_bingo = pd.DataFrame(columnas, dtype=str).transpose()

# Asignar los encabezados de las columnas
tabla_bingo.columns = ['B', 'I', 'N', 'G', 'O']

# Marcar la casilla central como 'libre'
tabla_bingo.iloc[2, 2] = 'libre'

# Mostrar la tabla de bingo
print(tabla_bingo)

    B   I      N   G   O
0   2  16     31  46  61
1   5  22     36  48  62
2   7  24  libre  50  72
3   9  25     43  56  73
4  10  30     45  58  74
